In [2]:
#Importamos librerias
from openpyxl import load_workbook
import os
from google.cloud import bigquery
import datetime
import pandas as pd
import numpy as np
import re

In [3]:
#Funcion Query
def bq_consulta(sql):
    """
    función para hacer consulta de bigquery
    Parameters:
        sql (str): query para ejecutar consulta
    Returns:
        df (pandas.DataFrame): Tabla con los datos de la consulta.
    """
    client = bigquery.Client()
    df = client.query(sql).to_dataframe()
    return df

In [4]:
my_path = os.getcwd()
today = datetime.date.today()
last_week = today - datetime.timedelta(days=7)
fecha=(last_week.strftime("%d/%m/%Y"))

In [4]:
datos="""
WITH todo_pv AS (
SELECT afi_hash64 FROM `sura-inception.proteccion_co_zconfiable.afi_segmento_mes` 
WHERE EstadoPV = 'ACT'),

clientes_saldo AS (SELECT cuentas.afi_hash64, SUM(MSDNMVALTOTALTERNATIVA) SALDO_TOTAL
FROM `sura-inception.proteccion_co_zcarga.ORACLE_PREVCXN1_TCMV_CUENTA` 
INNER JOIN  `sura-inception.proteccion_co_zcarga.ORACLE_PREVCXN1_TCMV_MATRIZ_SALDOSDIARIOS` ON (CUEID = MSDIDCUENTA)
INNER JOIN  `sura-inception.proteccion_co_zcarga.ORACLE_PREVCXN1_TENE_ALTERNATIVA_INVERSION` ON (MSDIDALTERNATIVA = AISID)
INNER JOIN `sura-inception.proteccion_co_zcarga.SQLSERVER_BIPROTECCIONDW_AUX_VAUXCUENTASAFILIADOS` AS cuentas ON (CAST(NumeroCuenta AS float64) = CUEID)
INNER JOIN `sura-inception.proteccion_co_zconfiable.afi_segmento_mes` AS segmento ON 
(segmento.afi_hash64 = cuentas.afi_hash64)
INNER JOIN `sura-inception.proteccion_co_zconfiable.afi_ScoreCliente_sabana_campanias` AS campa ON (campa.afi_hash64 = segmento.afi_hash64)
INNER JOIN `sura-inception.proteccion_co_zcarga.SQLSERVER_BIPROTECCIONDW_DW_DIMPLANES` AS planes ON
(planes.idPlan = cuentas.idPlan)
INNER JOIN `sura-inception.proteccion_co_zcarga.SQLSERVER_BIPROTECCIONDW_DW_DIMSUBTIPOPLANES` AS subplanes ON
(subplanes.skSubTipoPlan = planes.fkSubTipoPlan)
AND FORMAT_DATE('%d/%m/%Y', MSDFESALDO) = "{}"
AND cuentas.descProducto = 'AHORRO VOLUNTARIO'
GROUP BY  afi_hash64),

clientes_perfilados AS (		
  SELECT
		afi_hash64, ps.name as descPerfil, c.updated_at as fechaPerfil,

    DATETIME_TRUNC(CURRENT_DATETIME, DAY) fecha_hoy,

    DATE_DIFF(DATETIME_TRUNC(CURRENT_DATETIME, DAY), CAST(c.updated_at AS DATETIME), month) vigencia_perfil_meses,

		FROM 
		`sura-inception.proteccion_co_zcarga.AWS_RDSPRODPOS001_PERFILINVCXN1_CURRENT_PROFILE` c, 
		`sura-inception.proteccion_co_zcarga.AWS_RDSPRODPOS001_PERFILINVCXN1_PROF_DET` pd, 
		`sura-inception.proteccion_co_zcarga.AWS_RDSPRODPOS001_PERFILINVCXN1_SRVVER_SRVPROF` ss, 
		`sura-inception.proteccion_co_zcarga.AWS_RDSPRODPOS001_PERFILINVCXN1_PRF_SRV` ps
		WHERE pd.curr_prof_id = c.id AND pd.srvver_srvprof_id = ss.id AND ps.id = ss.srv_prof_id
		AND ps.name IN ('Arriesgado', 'Cauteloso', 'Moderado', 'Muy Conservador', 'Valiente')),

  clientes_alternativas AS (
    SELECT DISTINCT cuentas.afi_hash64, subplanes.descSubTipoPlan,
FROM `sura-inception.proteccion_co_zcarga.ORACLE_PREVCXN1_TCMV_CUENTA` 
INNER JOIN  `sura-inception.proteccion_co_zcarga.ORACLE_PREVCXN1_TCMV_MATRIZ_SALDOSDIARIOS` ON (CUEID = MSDIDCUENTA)
INNER JOIN  `sura-inception.proteccion_co_zcarga.ORACLE_PREVCXN1_TENE_ALTERNATIVA_INVERSION` ON (MSDIDALTERNATIVA = AISID)
INNER JOIN `sura-inception.proteccion_co_zcarga.SQLSERVER_BIPROTECCIONDW_AUX_VAUXCUENTASAFILIADOS` AS cuentas ON (CAST(NumeroCuenta AS float64) = CUEID)
INNER JOIN `sura-inception.proteccion_co_zconfiable.afi_segmento_mes` AS segmento ON 
(segmento.afi_hash64 = cuentas.afi_hash64)
INNER JOIN `sura-inception.proteccion_co_zconfiable.afi_ScoreCliente_sabana_campanias` AS campa ON (campa.afi_hash64 = segmento.afi_hash64)
INNER JOIN `sura-inception.proteccion_co_zcarga.SQLSERVER_BIPROTECCIONDW_DW_DIMPLANES` AS planes ON
(planes.idPlan = cuentas.idPlan)
INNER JOIN `sura-inception.proteccion_co_zcarga.SQLSERVER_BIPROTECCIONDW_DW_DIMSUBTIPOPLANES` AS subplanes ON
(subplanes.skSubTipoPlan = planes.fkSubTipoPlan)
AND FORMAT_DATE('%d/%m/%Y', MSDFESALDO) = "{}"
AND cuentas.descProducto = 'AHORRO VOLUNTARIO'
AND subplanes.descSubTipoPlan = 'ALTERNATIVAS CERRADAS'
--AND CUEIDPRODUCTO = 1
GROUP BY  afi_hash64, subplanes.descSubTipoPlan)

SELECT todo_pv.afi_hash64,
CASE 
   WHEN clientes_saldo.afi_hash64 IS NULL THEN 'no tiene saldo'
   WHEN clientes_saldo.afi_hash64 IS NOT NULL THEN 'con saldo'
END afi_saldo,
descPerfil, fechaPerfil, vigencia_perfil_meses,
CASE 
   WHEN clientes_perfilados.descPerfil IS NULL THEN 'no tiene perfil'
   WHEN clientes_perfilados.descPerfil IS NOT NULL THEN 'tiene perfil'
END cliente_perfil,

 CASE
    WHEN vigencia_perfil_meses > 24 THEN 'desactualizado'
    WHEN vigencia_perfil_meses <= 24 THEN 'actualizado'
END
  estado_perfil,

 CASE
    WHEN descSubTipoPlan IS NULL THEN 'no tiene AC'
    WHEN vigencia_perfil_meses IS NOT NULL THEN 'tiene AC'
END
  clientes_ac,

FROM todo_pv
LEFT JOIN clientes_saldo USING (afi_hash64)
LEFT JOIN clientes_perfilados USING (afi_hash64)
LEFT JOIN clientes_alternativas USING (afi_hash64)
""".format(fecha,fecha)

In [21]:
GP="""
SELECT * FROM `sura-inception.proteccion_co_zconfiable.Afi_Segmento_2023_V2` 
WHERE segmento="GP"
"""

In [22]:
base_GP=bq_consulta(GP)

/home/jjvelez@PROTECCION.LOCAL/.virtualenvs/[Suitjjvm]/lib/python3.7/site-packages/google/auth/_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/home/jjvelez@PROTECCION.LOCAL/.virtualenvs/[Suitjjvm]/lib/python3.7/site-packages/google/auth/_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [7]:
conveniencia="""
SELECT * FROM `pro-cientificos.proteccion_co_sbx_ahorro_inversion.resul_suitability_adj` 
"""

In [8]:

suitab_conv=bq_consulta(conveniencia)


/home/jjvelez@PROTECCION.LOCAL/.virtualenvs/[Suitjjvm]/lib/python3.7/site-packages/google/auth/_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/home/jjvelez@PROTECCION.LOCAL/.virtualenvs/[Suitjjvm]/lib/python3.7/site-packages/google/auth/_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [36]:
#base_GP.head()
base_GP.to_excel("CLIENTESG.xlsx")

#suitab_conv=suitab_conv.set_index("afi_hash64")



In [34]:
result= pd.merge(suitab_conv,base_GP,how='left',left_on="afi_hash64",right_on="afi_hash64")
#result=result.set_index("afi_hash64")
#result_GP=pd.merge(result,dataframe_excel["CLIENTE GP"],how='left',left_index=True,right_index=True)

In [14]:
#result_GP=pd.merge(result,dataframe_excel["CLIENTE GP"],how='left',left_index=True,right_index=True)

In [35]:
result

,afi_hash64,Segmento_Actual,CicloVida,CiudadUbicacion,DptoUbicacion,perfil_de_riesgo,FECHA_SALDO,saldo_universal,SALDO_TOTAL,coefsobre,suitability,segmento,detalle,subsegmento
0,273d42a4b134fdabd6af0d65f5b3b94434498df7,2.PRIME,SIN INFORMACION,CALI,VALLE DEL CAUCA,Cauteloso,11/05/2023,1.077254e+08,1.077254e+08,0.350670,sobreexpuesto,NaN,NaN,NaN
1,3c89ba41feeb96df65a236de012aeb92c306e436,3.ALTO VALOR,SIN INFORMACION,CALI,VALLE DEL CAUCA,Moderado,11/05/2023,2.652784e+07,2.652784e+07,0.000000,ajustado,NaN,NaN,NaN
2,66c417ceb61ce4cbfcaae4639d8fd18afe7324cb,4.RENTAS MEDIAS A,SIN INFORMACION,CALI,VALLE DEL CAUCA,Muy Conservador,11/05/2023,8.936733e+07,8.936733e+07,0.000000,ajustado,NaN,NaN,NaN
3,6892ae126b62b54df1d5ef3b3672ba7a831cb269,6.RENTAS MASIVAS C,SIN INFORMACION,CALI,VALLE DEL CAUCA,Moderado,11/05/2023,4.018505e+08,4.018505e+08,0.750000,sobreexpuesto,GP,Es alguien con un saldo superior a 200 millones,GP-BAJO
4,6a015ed539ea5ffabe36d8bd7413d842b7617bef,3.ALTO VALOR,SIN INFORMACION,CALI,VALLE DEL CAUCA,Cauteloso,11/05/2023,1.208263e+08,1.208263e+08,0.000000,ajustado,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56907,267ab8f5dbde4c2bf0f90d4d66a25b6256f358ad,1.ALTO PATRIMONIO,ADULTO MAYOR ETAPA 2 (HOMBRE),GUADALAJARA DE BUGA,VALLE DEL CAUCA,Muy Conservador,11/05/2023,1.029791e+09,1.029791e+09,0.018526,sobreexpuesto,GP,Es alguien con un saldo superior a 200 millones,GP-MEDIO
56908,fad0133902d611fd5040b227e27511662f3c4d7b,2.PRIME,ADULTO MAYOR ETAPA 2 (HOMBRE),GUADALAJARA DE BUGA,VALLE DEL CAUCA,Arriesgado,11/05/2023,1.968340e+08,1.968340e+08,0.000000,ajustado,GP,Es alguien con un saldo superior a 200 millones,GP-BAJO
56909,940b8310a3e5e296307b5d98500cb973a0550c1f,4.RENTAS MEDIAS A,ADULTO MAYOR ETAPA 2 (HOMBRE),SANTA ROSA DE CABAL,RISARALDA,Moderado,11/05/2023,1.536130e+08,1.536130e+08,0.000000,ajustado,NaN,NaN,NaN
56910,d389afcf8f1dbcaeab3a48c2a2b1ef9f840194e5,2.PRIME,ADULTO MAYOR ETAPA 2 (HOMBRE),SANTA ROSA DE CABAL,RISARALDA,Cauteloso,11/05/2023,3.981130e+08,3.981130e+08,0.000000,ajustado,GP,Es alguien con un saldo superior a 200 millones,GP-BAJO


In [16]:
#result_GP.dtypes
result_GP['fechaPerfil'] = result_GP['fechaPerfil'].dt.tz_localize(None)

In [53]:
result_GP.to_excel("Monitoreo_clientes_Gp.xlsx")

In [5]:
sin_saldo_no_suit="""
with saldos as (

SELECT cuentas.afi_hash64, SUM(MSDNMVALTOTALTERNATIVA) SALDO_TOTAL

,SUM(MSDNMVALTOTALTERNAUNIDADES) UNIDADES_TOTALES, FORMAT_DATE('%d/%m/%Y', MSDFESALDO) AS FECHA_SALDO,

FROM `sura-inception.proteccion_co_zcarga.ORACLE_PREVCXN1_TCMV_CUENTA`

INNER JOIN  `sura-inception.proteccion_co_zcarga.ORACLE_PREVCXN1_TCMV_MATRIZ_SALDOSDIARIOS` ON (CUEID = MSDIDCUENTA)

INNER JOIN  `sura-inception.proteccion_co_zcarga.ORACLE_PREVCXN1_TENE_ALTERNATIVA_INVERSION` ON (MSDIDALTERNATIVA = AISID)

INNER JOIN `sura-inception.proteccion_co_zcarga.SQLSERVER_BIPROTECCIONDW_AUX_VAUXCUENTASAFILIADOS` AS cuentas ON (CAST(NumeroCuenta AS float64) = CUEID)

INNER JOIN `sura-inception.proteccion_co_zconfiable.afi_segmento_mes` AS segmento ON

(segmento.afi_hash64 = cuentas.afi_hash64)

INNER JOIN `sura-inception.proteccion_co_zconfiable.afi_ScoreCliente_sabana_campanias` AS campa ON (campa.afi_hash64 = segmento.afi_hash64)

INNER JOIN `sura-inception.proteccion_co_zcarga.SQLSERVER_BIPROTECCIONDW_DW_DIMPLANES` AS planes ON

(planes.idPlan = cuentas.idPlan)

INNER JOIN `sura-inception.proteccion_co_zcarga.SQLSERVER_BIPROTECCIONDW_DW_DIMSUBTIPOPLANES` AS subplanes ON

(subplanes.skSubTipoPlan = planes.fkSubTipoPlan)

AND FORMAT_DATE('%d/%m/%Y', MSDFESALDO) = '23/05/2023'

AND cuentas.descProducto = 'AHORRO VOLUNTARIO'

--AND CUEIDPRODUCTO = 1

GROUP BY  afi_hash64, Msdfesaldo)


select * from saldos

right join `pro-cientificos.proteccion_co_sbx_ahorro_inversion.data_afi_suita` USING (afi_hash64)

WHERE SALDO_TOTAL IS NULL
--where afi_hash64 = 'ff2e8f6d7ba141f152ed1d1cb46705f3830636c4'
"""

sin_saldo=bq_consulta(sin_saldo_no_suit)

/home/jjvelez@PROTECCION.LOCAL/.virtualenvs/[Suitjjvm]/lib/python3.7/site-packages/google/auth/_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/home/jjvelez@PROTECCION.LOCAL/.virtualenvs/[Suitjjvm]/lib/python3.7/site-packages/google/auth/_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [7]:
sin_perfil_no_suit="""
WITH
  perfil AS (
  SELECT
    afi_hash64,
    ps.name AS descPerfil,
    c.updated_at AS fechaPerfil,
    DATETIME_TRUNC(CURRENT_DATETIME, DAY) fecha_hoy,
    DATE_DIFF(DATETIME_TRUNC(CURRENT_DATETIME, DAY), CAST(c.updated_at AS DATETIME), month) vigencia_perfil_meses,
  FROM
    `sura-inception.proteccion_co_zcarga.AWS_RDSPRODPOS001_PERFILINVCXN1_CURRENT_PROFILE` c,
    `sura-inception.proteccion_co_zcarga.AWS_RDSPRODPOS001_PERFILINVCXN1_PROF_DET` pd,
    `sura-inception.proteccion_co_zcarga.AWS_RDSPRODPOS001_PERFILINVCXN1_SRVVER_SRVPROF` ss,
    `sura-inception.proteccion_co_zcarga.AWS_RDSPRODPOS001_PERFILINVCXN1_PRF_SRV` ps
  WHERE
    pd.curr_prof_id = c.id
    AND pd.srvver_srvprof_id = ss.id
    AND ps.id = ss.srv_prof_id
    AND ps.name IN ('Arriesgado',
      'Cauteloso',
      'Moderado',
      'Muy Conservador',
      'Valiente') )
SELECT
  *
FROM
  perfil
RIGHT JOIN
  `pro-cientificos.proteccion_co_sbx_ahorro_inversion.data_afi_suita`
USING
  (afi_hash64)
WHERE
  descPerfil IS NULL
  --where afi_hash64 = 'ff2e8f6d7ba141f152ed1d1cb46705f3830636c4'
"""

sin_perfil=bq_consulta(sin_perfil_no_suit)

/home/jjvelez@PROTECCION.LOCAL/.virtualenvs/[Suitjjvm]/lib/python3.7/site-packages/google/auth/_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/home/jjvelez@PROTECCION.LOCAL/.virtualenvs/[Suitjjvm]/lib/python3.7/site-packages/google/auth/_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [5]:
desicion_patroci="""
WITH
  planes AS (
  SELECT
    cuentas.afi_hash64,
    --subplanes.descSubTipoPlan,
    --SUM(MSDNMVALTOTALTERNATIVA) SALDO_TOTAL,
    FORMAT_DATE('%d/%m/%Y', MSDFESALDO) AS FECHA_SALDO,
    PRODSRESPONSABLERECOMPOSICION
  FROM
    `sura-inception.proteccion_co_zcarga.ORACLE_PREVCXN1_TCMV_CUENTA`
  INNER JOIN
    `sura-inception.proteccion_co_zcarga.ORACLE_PREVCXN1_TCMV_MATRIZ_SALDOSDIARIOS`
  ON
    (CUEID = MSDIDCUENTA)
  INNER JOIN
    `sura-inception.proteccion_co_zcarga.ORACLE_PREVCXN1_TENE_ALTERNATIVA_INVERSION`
  ON
    (MSDIDALTERNATIVA = AISID)
  INNER JOIN
    `sura-inception.proteccion_co_zcarga.SQLSERVER_BIPROTECCIONDW_AUX_VAUXCUENTASAFILIADOS` AS cuentas
  ON
    (CAST(NumeroCuenta AS float64) = CUEID)
  INNER JOIN
    `sura-inception.proteccion_co_zconfiable.afi_segmento_mes` AS segmento
  ON
    (segmento.afi_hash64 = cuentas.afi_hash64)
  INNER JOIN
    `sura-inception.proteccion_co_zcarga.SQLSERVER_BIPROTECCIONDW_DW_DIMPLANES` AS planes
  ON
    (planes.idPlan = cuentas.idPlan)
  INNER JOIN
    `sura-inception.proteccion_co_zcarga.SQLSERVER_BIPROTECCIONDW_DW_DIMSUBTIPOPLANES` AS subplanes
  ON
    (subplanes.skSubTipoPlan = planes.fkSubTipoPlan)
  INNER JOIN
    `sura-inception.proteccion_co_zcarga.ORACLE_PREVCXN1_TENE_PRODUCTO` AS planes_distri
  ON
    (CAST(PROID AS STRING) = cuentas.idPlan)
    AND FORMAT_DATE('%d/%m/%Y', MSDFESALDO) = "23/05/2023"
    AND cuentas.descProducto = 'AHORRO VOLUNTARIO'
  WHERE
    subplanes.descSubTipoPlan IN ('PLAN INSTITUCIONAL',
      'PLAN INSTITUCIONAL CORPORATIVO')
    AND PRODSRESPONSABLERECOMPOSICION IN ('PATROCINADOR')
  GROUP BY
    afi_hash64,
    --subplanes.descSubTipoPlan,
    Msdfesaldo,
    PRODSRESPONSABLERECOMPOSICION )
SELECT
  *
FROM
  planes
RIGHT JOIN
  `pro-cientificos.proteccion_co_sbx_ahorro_inversion.data_afi_suita`
USING
  (afi_hash64)
  --where PRODSRESPONSABLERECOMPOSICION is not null

"""

patrocinadora=bq_consulta(desicion_patroci)

/home/jjvelez@PROTECCION.LOCAL/.virtualenvs/[Suitjjvm]/lib/python3.7/site-packages/google/auth/_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/home/jjvelez@PROTECCION.LOCAL/.virtualenvs/[Suitjjvm]/lib/python3.7/site-packages/google/auth/_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [ ]:
sin_saldo.to_excel("sin_saldo_nosuit.xlsx")
sin_perfil.to_excel("sin_perfil_nosuit.xlsx")
patrocinadora.to_excel("patrocinadora_nosuit.xlsx")

In [ ]:
patrocinadora.to_excel("patrocinadora_nosuit.xlsx")

In [5]:
vencimientos="""
SELECT AISID, AISFEINICIOVIGENCIA,  AISFEFINVIGENCIA, AISFECREACION, AISDSNOMBRECORTALTERNAINVERS, AISDSNOMBRELARGALTERNAINVERS FROM `sura-inception.proteccion_co_zcarga.ORACLE_PREVCXN1_TENE_ALTERNATIVA_INVERSION`

"""

vencimientos_AC=bq_consulta(vencimientos)

/home/jjvelez@PROTECCION.LOCAL/.virtualenvs/[Suitjjvm]/lib/python3.7/site-packages/google/auth/_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/home/jjvelez@PROTECCION.LOCAL/.virtualenvs/[Suitjjvm]/lib/python3.7/site-packages/google/auth/_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [8]:
vencimientos_AC.dtypes
vencimientos_AC['AISFEINICIOVIGENCIA'] = vencimientos_AC['AISFEINICIOVIGENCIA'].dt.tz_localize(None)
vencimientos_AC['AISFEFINVIGENCIA'] = vencimientos_AC['AISFEFINVIGENCIA'].dt.tz_localize(None)
vencimientos_AC['AISFECREACION'] = vencimientos_AC['AISFECREACION'].dt.tz_localize(None)
vencimientos_AC.to_excel("vencimientos_AC.xlsx")